# Data Crawling NO2

Proses ini digunakan untuk mengunduh dan memproses data NO2 (Nitrogen Dioxide) dari satelit Sentinel-5P menggunakan OpenEO API.

Import library dan koneksi ke Copernicus Data Space Ecosystem

In [ ]:
import openeo
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import xarray as xr

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()
print("Connected to Copernicus Data Space Ecosystem")

Definisi Area of Interest (AOI) wilayah Sampang dan parameter waktu (2021-2025)

In [ ]:
aoi = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              113.23789711828073,
              -7.185910697549147
            ],
            [
              113.23789711828073,
              -7.207774079752852
            ],
            [
              113.26271944187818,
              -7.207774079752852
            ],
            [
              113.26271944187818,
              -7.185910697549147
            ],
            [
              113.23789711828073,
              -7.185910697549147
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}

# Definisi extent spasial dari koordinat AOI
spatial_extent = {
    "west": 113.23789711828073,
    "south": -7.207774079752852,
    "east": 113.26271944187818,
    "north": -7.185910697549147
}

start_date = "2021-01-01"
end_date = "2025-10-19"

print(f"AOI defined for coordinates: {spatial_extent}")
print(f"Time range: {start_date} to {end_date}")
print("Setup completed successfully")

Loading data Sentinel-5P NO2 dan agregasi temporal harian

In [ ]:
print("Loading Sentinel-5P NO2 data...")

s5p_no2 = connection.load_collection(
    "SENTINEL_5P_L2",
    temporal_extent=[start_date, end_date],
    spatial_extent=spatial_extent,
    bands=["NO2"],
)

s5p_monthly = s5p_no2.aggregate_temporal_period(
    period="day",
    reducer="mean"
)

print("Data collection and aggregation configured successfully")

Eksekusi job batch processing dan export ke file NetCDF

In [ ]:
print("Starting data processing job...")

job = s5p_monthly.execute_batch(
    title="NO2 Averages 2021-2025", 
    outputfile="no2_averages_4years.nc"
)